In [13]:
!pip install langgraph langchain-groq duckduckgo-search

In [12]:
!pip install langchain_community

In [11]:
%pip install -qU duckduckgo-search langchain-community

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
from typing import List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.pydantic_v1 import BaseModel, Field
import asyncio

# Currency conversion tool using LLM extraction
class CurrencyConversionTool(BaseTool):
    name: str = Field(default="currency_converter", description="Name of the tool")
    description: str = Field(
        default="Converts prices between different currencies using web search and an LLM for extraction",
        description="Tool description"
    )

    def _run(self, currency: str) -> float:
        return self._convert_currency(currency)

    def _convert_currency(self, currency: str) -> float:
        # Use DuckDuckGo search to find exchange rate text
        search_tool = DuckDuckGoSearchRun()
        search_query = f"{currency} to USD exchange rate"
        search_results = search_tool.run(search_query)
        print(search_results)
        # Initialize LLM to extract the exchange rate from the search results.
        llm = ChatGroq(
            groq_api_key="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",
            temperature=0,
            model_name="llama3-70b-8192"
        )
        prompt = (
            f"Extract the numeric exchange rate for converting 1 {currency} to USD "
            f"from the following text. Only provide the number:\n\n{search_results}"
        )
        # Use direct call instead of llm.run()
        llm_output = llm(prompt)
        try:
            rate = float(llm_output.strip())
            return rate
        except Exception as e:
            raise ValueError(f"Could not extract conversion rate for {currency}. LLM output: {llm_output}") from e

# Calculator tool
class CalculatorTool(BaseTool):
    name: str = "calculator"
    description: str = "Performs basic arithmetic calculations"

    def _run(self, expression: str) -> float:
        # WARNING: eval can be dangerous in production environments.
        return eval(expression)

# State for the agent
class AgentState(BaseModel):
    input: List[Dict[Any, Any]] = Field(default_factory=list)
    output: List[Dict[Any, Any]] = Field(default_factory=list)
    currency_rates: Dict[str, float] = Field(default_factory=dict)

# Create the agent workflow
async def create_currency_conversion_agent():
    # Initialize tools
    currency_tool = CurrencyConversionTool()
    calculator_tool = CalculatorTool()

    # Initialize Groq LLM (used for agent integration; not directly in conversion now)
    llm = ChatGroq(
        groq_api_key="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",
        temperature=0,
        model_name="llama3-70b-8192"
    )

    # Define the workflow graph
    workflow = StateGraph(AgentState)

    # Node to fetch currency rates using LLM extraction
    async def fetch_currency_rates(state: AgentState):
        # Collect unique currencies (excluding USD) from the input
        currencies = set(item['currency'] for item in state.input if item['currency'] != 'USD')
        currency_rates = {}
        for currency in currencies:
            try:
                rate = currency_tool.run(currency)
                currency_rates[currency] = rate
            except Exception as e:
                print(f"Error fetching rate for {currency}: {e}")
        new_state = state.dict()
        new_state['currency_rates'] = currency_rates
        return new_state

    # Node to convert prices using the fetched currency rates and the calculator tool
    async def convert_prices(state: AgentState):
        converted_results = []
        for item in state.input:
            converted_item = item.copy()
            if item['currency'] != 'USD':
                try:
                    rate = state.currency_rates.get(item['currency'])
                    if rate:
                        # Use the calculator tool to compute price * rate via an expression string
                        expression = f"{item['price']}*{rate}"
                        calculated_value = calculator_tool.run(expression)
                        converted_item['price'] = round(float(calculated_value), 2)
                        converted_item['currency'] = 'USD'
                except Exception as e:
                    print(f"Error converting price for {item.get('product_name')}: {e}")
            converted_results.append(converted_item)
        new_state = state.dict()
        new_state['output'] = converted_results
        return new_state

    # Add nodes to the workflow
    workflow.add_node("fetch_rates", fetch_currency_rates)
    workflow.add_node("convert_prices", convert_prices)

    # Define edges: first fetch rates, then convert prices, then end
    workflow.set_entry_point("fetch_rates")
    workflow.add_edge("fetch_rates", "convert_prices")
    workflow.add_edge("convert_prices", END)

    # Compile the graph
    app = workflow.compile()
    return app

# Main execution function
async def main():
    # Input JSON data (demo)
    input_data = {
        "results": [
            {
                "product_name": "Sony, 55 Inch, 4K HDR, Android TV",
                "price": 2499,
                "currency": "SAR",
                "source": "https://www.extra.com/en-sa/electronics/television/led/sony-55-inch-4k-hdr-android-tv/p/100312894",
                "vat_status": "Incl. VAT",
                "payment_type": None,
                "features_of_product": "3 HDMI inputs, Bluetooth, 4K Processor X1, 4K X-Reality PRO, Motionflow XR, Clear Phase, Chromecast built-in, X Protection PRO, Android TV, 2 USB inputs, 16GB internal memory, 10W x 2 speakers, 50Hz, 4K HDR, LED display, 55 inches, 2022 model",
                "vendor_name": None,
                "customer_rating": "4.5"
            },
            {
                "product_name": "Sony Bravia 55 inch Google K-55S30 4k LED TV",
                "price": 3199,
                "currency": "SAR",
                "source": "https://www.almanea.sa/en/product/sony-bravia-55-inch-google-k-55s30-4k-led-tv-p-139904699455048?srsltid=AfmBOoopG0PLoKZaN4hs7CTlWE7L4_V44N6mO8STn8EGHCyQ4uQoh2qK",
                "vat_status": "after vat",
                "payment_type": None,
                "features_of_product": "4K - LED, 55 inches, UHD 4K, 2 USB Socket, 4 HDMI Sockets, Dimension Net (WxDxH): 123.3 * 7.2 * 71.3, Resolution (MP): 3840*2160, TV ANDROID: Yes, Warranty: 2 years, Country of Manufacture: Malaysia",
                "vendor_name": None,
                "customer_rating": None
            },
            {
                "product_name": "SONY Bravia 139 cm (55 inch) Ultra HD (4K) LED Smart TV (KD-55X85J)",
                "price": 79990,
                "currency": "INR",
                "source": "https://www.flipkart.com/sony-bravia-139-cm-55-inch-ultra-hd-4k-led-smart-tv/p/itmb1de6db72ae33",
                "vat_status": "after vat",
                "payment_type": None,
                "features_of_product": "Supported Apps: Not specified, Resolution: Ultra HD (4K) 3840 x 2160 Pixels, Sound Output: 20 W, Refresh Rate: 60 Hz, HDMI: 4, USB: 2, Built In Wi-Fi: Yes, Launch Year: 2021, Number of Speakers: 2, Power Requirement: 220-240 V, Power Consumption: 0.5 W (Standby)",
                "vendor_name": "SameDayDeliveryBLR",
                "customer_rating": None
            }
        ]
    }

    # Create and run the agent
    agent = await create_currency_conversion_agent()
    result = await agent.ainvoke({"input": input_data["results"]})

    # Print and save the result
    output = {"results": result['output']}
    print(json.dumps(output, indent=2))
    with open('converted_prices.json', 'w') as f:
        json.dump(output, f, indent=2)

# Run the main function
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()  # Allow nested event loops
    asyncio.run(main())


Convert Saudi Riyal to United States Dollar with the latest exchange rate and historical data. See the fixed peg of 1 USD = 3.75 SAR and quick conversions for common amounts. Quickly and easily calculate foreign exchange rates with this free currency converter. Further Information Saudi Riyal - US-Dollar. Close: 0.2666: Open: 0.2666: Daily Low: 0.2666: Daily High: 0. ... Find out the current and historical exchange rate of SAR to USD with this free currency calculator. See the rates table, graphs and monthly average for Aug 08, 2024. Learn the value of 1 Saudi Riyal (SAR) in United States Dollars (USD) today. The dynamics of the exchange rate change for a week, for a month, for a year on the chart and in the tables. Convert 1 Riyal to Dollars with an online currency converter. It is possible to convert any amount from Saudi Arabian Riyal to US Dollar (SAR to $) using Myfin currency converter at the live rates of 09:20 AM 02-28-2025. 1 SAR ; 100 SAR ... Historical data of the 1 Saudi Ar

In [10]:
!pip install langchain_experimental

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper


In [9]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

x=search.invoke("Bdt to RM exchange rate")

from langchain_core.pydantic_v1 import BaseModel, Field

class CurrencyConversion(BaseModel):
    from_currency: str = Field(..., description="The currency to convert from (e.g., USD)")
    to_currency: str = Field(..., description="The target currency to convert to (e.g., BDT)")
    conversion_rate: float = Field(..., description="The conversion rate from 1 unit of the 'from_currency' to the 'to_currency' (e.g., if 1 USD = 120 BDT, then conversion_rate is 120)")
model = ChatGroq(
        groq_api_key="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",
        temperature=0,
        model_name="llama3-8b-8192"
    )
structured_llm = model.with_structured_output(CurrencyConversion)
structured_llm.invoke(x)

ImportError: Could not import duckduckgo-search python package. Please install it with `pip install -U duckduckgo-search`.

In [ ]:
from langchain_groq import ChatGroq

CurrencyConversion(from_currency='MYR', to_currency='BDT', conversion_rate=27.4155)

In [1]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="https://cq1pjldgq1oqaf-11434.proxy.runpod.net",
    model="qwen2.5:14b",
    temperature=0,
    # other params...
)

In [2]:
from langchain import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool, initialize_agent


math_chain = LLMMathChain.from_llm(llm=llm)

# Wrap the math chain as a tool
calculator_tool = Tool.from_function(
    name="Calculator",
    func=math_chain.run,
    description="Useful for when you need to answer questions about math. Only math expressions should be passed."
)

# Initialize an agent with the calculator tool (using a zero-shot ReAct agent)
agent = initialize_agent(
    tools=[calculator_tool],
    llm=llm,  # Still use Groq for the agent
    agent="zero-shot-react-description",
    verbose=True
)

# Example query that requires calculation
query = "What is (4.5*2.1)^2.2?"
result = agent.run(query)
print(result)

C:\Users\Nilab\AppData\Local\Temp\ipykernel_20732\3446431988.py:16: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\Nilab\AppData\Local\Temp\ipykernel_20732\3446431988.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)




> Entering new AgentExecutor chain...
I need to calculate the expression (4.5*2.1)^2.2.
Action: Calculator
Action Input: (4.5*2.1)**2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: The result of (4.5*2.1)^2.2 is approximately 139.94.

> Finished chain.
The result of (4.5*2.1)^2.2 is approximately 139.94.


In [22]:
from langchain import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool, initialize_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool  # for the @tool decorator

# -------------------------------
# Calculator Tool Setup


# Create a math chain that acts as a calculator
math_chain = LLMMathChain.from_llm(llm=llm)

# Wrap the math chain as a tool; the description guides the agent to use this tool for math expressions.
calculator_tool = Tool.from_function(
    name="Calculator",
    func=math_chain.run,
    description="Useful for when you need to answer questions about math. Only math expressions should be passed."
)

# -------------------------------
# Currency Conversion Tool Setup
# -------------------------------
# Define the structured output model for currency conversion
class CurrencyConversion(BaseModel):
    from_currency: str = Field(..., description="The currency to convert from (e.g., USD)")
    to_currency: str = Field(..., description="The target currency to convert to (e.g., BDT)")
    conversion_rate: float = Field(
        ...,
        description="The conversion rate from 1 unit of the 'from_currency' to the 'to_currency."
    )

# Wrap the conversion logic as a tool using the @tool decorator.
@tool("currency-conversion-tool")
def currency_conversion_tool(query: str) -> CurrencyConversion:
    """
    This tool takes a query (e.g., "Bdt to RM exchange rate") and returns a structured
    CurrencyConversion object containing the from_currency, to_currency, and the conversion rate.
    """
    # Perform a DuckDuckGo search to retrieve exchange rate information.
    search = DuckDuckGoSearchRun()
    search_result = search.invoke(query)

    # Initialize the ChatGroq model with structured output for CurrencyConversion.
    structured_llm = llm.with_structured_output(CurrencyConversion)

    # Process the search result to obtain a CurrencyConversion output.
    conversion = structured_llm.invoke(search_result)
    return conversion

# -------------------------------
# Agent Initialization with Both Tools
# -------------------------------
# Add both the Calculator and Currency Conversion tools to the agent.
tools = [calculator_tool, currency_conversion_tool]

# Initialize a zero-shot ReAct agent with the two tools.
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


# Query that triggers the Currency Conversion tool.
conversion_query = "1 inr to usd exchange rate"
conversion_result = agent.run(conversion_query)
print("Currency Conversion Result:", conversion_result)




> Entering new AgentExecutor chain...
I need to find out the current exchange rate from INR (Indian Rupee) to USD (US Dollar).
Action: currency-conversion-tool
Action Input: "1 inr to usd"
Observation: from_currency='INR' to_currency='USD' conversion_rate=0.0114393
Thought:I now know the final answer.
Final Answer: The current exchange rate is 0.0114393 USD for 1 INR.

> Finished chain.
Currency Conversion Result: The current exchange rate is 0.0114393 USD for 1 INR.


In [15]:
# Improved prompt example:
prompt = (
    "You are given exchange rate information. "
    "Please extract the conversion details and output them strictly in JSON format following the template below:\n\n"
    "{\n"
    "  \"from_currency\": \"<string - the source currency, e.g., 'SAR'>\",\n"
    "  \"to_currency\": \"<string - the target currency, e.g., 'USD'>\",\n"
    "  \"conversion_rate\": <number - the conversion rate from 'from_currency' to 'to_currency'>\n"
    "}\n\n"
    "Make sure the JSON is valid and does not include any additional text."
)

# Then, call your language model with the improved prompt:
response_content = llm.invoke(prompt + conversion_result).content
print(response_content)


Based on your instruction and the provided example rate, while excluding any unneeded information and focusing just on accurately formatting the output as pure JSON, here's how we would represent today's INR to USD exchange rate using the given template:

```json
{
  "from_currency": "INR",
  "to_currency": "USD",
  "conversion_rate": 0.012
}
```

Please note that for actual use, always verify with a reliable and current source since exchange rates fluctuate regularly.


In [25]:
import json

# Sample product data
products = {
    "results": [
        {
            "product_name": "Sony, 55 Inch, 4K HDR, Android TV",
            "price": 2499,
            "currency": "SAR",
            "source": "https://www.extra.com/en-sa/electronics/television/led/sony-55-inch-4k-hdr-android-tv/p/100312894",
            "vat_status": "Incl. VAT",
            "payment_type": None,
            "features_of_product": "3 HDMI inputs, Bluetooth, 4K Processor X1, 4K X-Reality PRO, Motionflow XR, Clear Phase, Chromecast built-in, X Protection PRO, Android TV, 2 USB inputs, 16GB internal memory, 10W x 2 speakers, 50Hz, 4K HDR, LED display, 55 inches, 2022 model",
            "vendor_name": None,
            "customer_rating": "4.5"
        },
        {
            "product_name": "Sony Bravia 55 inch Google K-55S30 4k LED TV",
            "price": 3199,
            "currency": "SAR",
            "source": "https://www.almanea.sa/en/product/sony-bravia-55-inch-google-k-55s30-4k-led-tv-p-139904699455048?srsltid=AfmBOoopG0PLoKZaN4hs7CTlWE7L4_V44N6mO8STn8EGHCyQ4uQoh2qK",
            "vat_status": "after vat",
            "payment_type": None,
            "features_of_product": "4K - LED, 55 inches, UHD 4K, 2 USB Socket, 4 HDMI Sockets, Dimension Net (WxDxH): 123.3 * 7.2 * 71.3, Resolution (MP): 3840*2160, TV ANDROID: Yes, Warranty: 2 years, Country of Manufacture: Malaysia",
            "vendor_name": None,
            "customer_rating": None
        },
        {
            "product_name": "SONY Bravia 139 cm (55 inch) Ultra HD (4K) LED Smart TV (KD-55X85J)",
            "price": 79990,
            "currency": "INR",
            "source": "https://www.flipkart.com/sony-bravia-139-cm-55-inch-ultra-hd-4k-led-smart-tv/p/itmb1de6db72ae33",
            "vat_status": "after vat",
            "payment_type": None,
            "features_of_product": "Supported Apps: Not specified, Resolution: Ultra HD (4K) 3840 x 2160 Pixels, Sound Output: 20 W, Refresh Rate: 60 Hz, HDMI: 4, USB: 2, Built In Wi-Fi: Yes, Launch Year: 2021, Number of Speakers: 2, Power Requirement: 220-240 V, Power Consumption: 0.5 W (Standby)",
            "vendor_name": "SameDayDeliveryBLR",
            "customer_rating": None
        }
    ]
}

# Cache to store conversion rates so that we only query once per currency.
conversion_cache = {}

# Loop through each product in the dataset.
for product in products["results"]:
    # Only process products whose currency is not USD.
    if product["currency"].upper() != "USD":
        from_currency = product["currency"]
        # Check the cache first
        if from_currency not in conversion_cache:
            # Prepare a query for the agent, e.g., "SAR to USD exchange rate"
            conversion_query = f"Directly Search it first as it is.{from_currency} to USD exchange rate"
            # Call the agent to get the conversion rate.
            # It is expected that the agent uses your currency conversion tool and returns
            # a structured result containing a "conversion_rate" field.
            conversion_result = agent.run(conversion_query)
            conversion_result=llm.invoke(prompt + conversion_result).content
            # If the result is a JSON string, parse it.
            if isinstance(conversion_result, str):
                try:
                    conversion_data = json.loads(conversion_result)
                except json.JSONDecodeError:
                    raise ValueError("Failed to decode conversion result as JSON.")
            else:
                conversion_data = conversion_result

            # Extract the conversion rate from the structured output.
            conversion_rate = conversion_data.get("conversion_rate")
            if conversion_rate is None:
                raise ValueError(f"No conversion rate found in the response for {from_currency} to USD.")
            # Cache the conversion rate.
            conversion_cache[from_currency] = conversion_rate
        else:
            conversion_rate = conversion_cache[from_currency]

        # Update the product price by multiplying with the conversion rate.
        original_price = product["price"]
        product["price"] = original_price * conversion_rate
        # Update the currency field to USD.
        product["currency"] = "USD"

# Print the updated product list.
print(json.dumps(products, indent=4))




> Entering new AgentExecutor chain...
I need to find out the SAR to USD exchange rate.
Action: currency-conversion-tool
Action Input: "SAR to USD exchange rate"
Observation: from_currency='SAR' to_currency='USD' conversion_rate=0.2666
Thought:I now know the final answer.
Final Answer: The current exchange rate is 1 SAR = 0.2666 USD.

> Finished chain.


> Entering new AgentExecutor chain...
I need to find out the current INR to USD exchange rate.
Action: currency-conversion-tool
Action Input: "INR to USD exchange rate"
Observation: None
Thought:It seems there was an issue with retrieving the observation. I will attempt to get the correct conversion rate again.
Action: currency-conversion-tool
Action Input: "INR to USD exchange rate"
Observation: from_currency='USD' to_currency='INR' conversion_rate=87.21
Thought:It appears that there was a misunderstanding in the observation, as it should have provided an INR to USD conversion rate but instead gave a USD to INR conversion rate with a